In [1]:
import copy
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['LOCAL_RANK'] = '0'
import os.path as osp
import time

import mmcv
import torch
from mmcv.cnn.utils import revert_sync_batchnorm
from mmcv.utils import Config, get_git_hash

from mmseg import __version__
from mmseg.apis import set_random_seed, train_segmentor
from mmseg.datasets import build_dataset
from mmseg.models import build_segmentor
from mmseg.utils import (get_device, get_root_logger, setup_multi_processes)

import pvt, pvtv2

C:\Users\nexty\anaconda3\envs\PVT\lib\site-packages\mmcv\__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


In [2]:
cfg_path = '../my_configs/fpn_pvtv2_b5_casia512_320k.py'
cfg = Config.fromfile(cfg_path)

In [3]:
if cfg.get('cudnn_benchmark', False):
    torch.backends.cudnn.benchmark = True

cfg.work_dir = '../work_dirs/casia512_320k/fpn_pvtv2_b5'
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
timestamp = time.strftime('%Y%m%d_%H%M%S', time.localtime())
log_file = osp.join(cfg.work_dir, f'{timestamp}.log')
logger = get_root_logger(log_file=log_file, log_level=cfg.log_level)
setup_multi_processes(cfg)
meta = dict()

distributed = False

logger.info(f'Distributed training: {distributed}')
logger.info(f'Config:\n{cfg.pretty_text}')

2023-07-29 00:14:49,777 - mmseg - INFO - OpenCV num_threads is `16
2023-07-29 00:14:49,777 - mmseg - INFO - Distributed training: False
2023-07-29 00:14:49,940 - mmseg - INFO - Config:
log_config = dict(
    interval=50,
    hooks=[
        dict(type='TextLoggerHook', by_epoch=False),
        dict(type='TensorboardLoggerHook')
    ])
dist_params = dict(backend='nccl')
log_level = 'INFO'
load_from = None
resume_from = None
workflow = [('train', 1)]
cudnn_benchmark = True
find_unused_parameters = True
norm_cfg = dict(type='SyncBN', requires_grad=True)
model = dict(
    type='EncoderDecoder',
    pretrained='pretrained/pvt_v2_b5.pth',
    backbone=dict(type='pvt_v2_b5', style='pytorch'),
    neck=dict(
        type='FPN',
        in_channels=[64, 128, 320, 512],
        out_channels=256,
        num_outs=4),
    decode_head=dict(
        type='FPNHead',
        in_channels=[256, 256, 256, 256],
        in_index=[0, 1, 2, 3],
        feature_strides=[4, 8, 16, 32],
        channels=128,
  

In [4]:
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)
cfg.device = get_device()
seed = cfg.seed
meta['seed'] = seed

In [5]:
model = build_segmentor(
        cfg.model,
        train_cfg=cfg.get('train_cfg'),
        test_cfg=cfg.get('test_cfg'))
model.init_weights()
if not distributed:
    model = revert_sync_batchnorm(model)
logger.info(model)


c:\users\nexty\desktop\pvtv2-seg\mmsegmentation\mmseg\models\decode_heads\decode_head.py:120: UserWarning: threshold is not defined for binary, and defaultsto 0.3
  warnings.warn('threshold is not defined for binary, and defaults'
c:\users\nexty\desktop\pvtv2-seg\mmsegmentation\mmseg\models\losses\cross_entropy_loss.py:235: UserWarning: Default ``avg_non_ignore`` is False, if you would like to ignore the certain label and average loss over non-ignore labels, which is the same with PyTorch official cross_entropy, set ``avg_non_ignore=True``.
  warnings.warn(
2023-07-29 00:14:51,280 - mmseg - INFO - initialize FPN with init_cfg {'type': 'Xavier', 'layer': 'Conv2d', 'distribution': 'uniform'}
2023-07-29 00:14:51,290 - mmseg - INFO - initialize FPNHead with init_cfg {'type': 'Normal', 'std': 0.01, 'override': {'name': 'conv_seg'}}
2023-07-29 00:14:51,298 - mmseg - INFO - EncoderDecoder(
  (backbone): pvt_v2_b5(
    (patch_embed1): OverlapPatchEmbed(
      (proj): Conv2d(3, 64, kernel_size=

In [6]:
datasets = [build_dataset(cfg.data.train)]
if len(cfg.workflow) == 2:
    val_dataset = copy.deepcopy(cfg.data.val)
    val_dataset.pipeline = cfg.data.train.pipeline
    datasets.append(build_dataset(val_dataset))
model.CLASSES = datasets[0].CLASSES

2023-07-29 00:14:51,410 - mmseg - INFO - Loaded 4084 images


In [7]:
if cfg.checkpoint_config is not None:
        # save mmseg version, config file content and class names in
        # checkpoints as meta data
    cfg.checkpoint_config.meta = dict(
        mmseg_version=f'{__version__}+{get_git_hash()[:7]}',
        config=cfg.pretty_text,
        CLASSES=datasets[0].CLASSES,
        PALETTE=datasets[0].PALETTE)

In [ ]:
# Обучение модели
train_segmentor(
        model,
        datasets,
        cfg,
        distributed=distributed,
        validate=True,
        timestamp=timestamp,
        meta=meta)

2023-07-29 00:14:51,952 - mmseg - INFO - Loaded 511 images
2023-07-29 00:14:51,952 - mmseg - INFO - Start running, host: hentaimean@Aorus-Elite, work_dir: C:\Users\nexty\Desktop\PVTv2-Seg\work_dirs\casia512_320k\fpn_pvtv2_b5
2023-07-29 00:14:51,952 - mmseg - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) PolyLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) TensorboardLoggerHook              
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) PolyLrUpdaterHook                  
(LOW         ) IterTimerHook                      
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) TensorboardLoggerHook              
 -------------------- 
before_train_iter:
(VERY_HIGH   ) PolyLrUpdaterHook                  
(LOW         ) IterTimerHook 